# Transfer learning 
In this notebook, we will use the follwoing tools:
- Tensorflow (using tensorflow.keras to make life easier)
- classification-models (a library of pre-trained models)

Our task is to transfer a model from classifying the imagenet dataset to a much smaller dataset of picturs of bees and ants. The dataset is called hymenoptera, and comes from one of the pytorch examples. We will be replicating the pytorch example quite closely, but in keras.

We are using transfer learning because we expect it will do much better on our small dataset than starting from scratch. Imagenet has 1.3 million examples across 1000 classes; our dataset has 120 examples across 2 classes. 120 images is a very small number when trying to train a CNN.


Our task is to take a model from the classification zoo and use a technique called transfer learning to apply it to the hymenoptera dataset.

We'll break it down into several steps.

First, lets download the dataset and unzip it

In [ ]:
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
!unzip -qo hymenoptera_data.zip

## Install some packages if we don't already have them
!pip3 install git+https://github.com/qubvel/classification_models.git

If we have a look, we should now have a folder called 'hymenoptera' in which we have sub-folders called 'train' and 'validation'. 

This is a very important step which has already been done for us. It is *imperative* that you split your dataset before you do anything. You should take out a portion of your data and leave it alone until you've finished training your models. This is the test set, sometimes called the validation set.

Splitting the dataset has already been done for us here, so we don't need to worry about it.

Let's have a look at one of the images in the training set. To load it, we will import imread from skimage, and to show it we will use matplotlib.

In [ ]:
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
anAnt = imread("./hymenoptera_data/train/ants/1030023514_aad5c608f9.jpg")
aBee = imread("./hymenoptera_data/train/bees/1092977343_cb42b38d62.jpg")

fig, axes = plt.subplots(1,2, figsize=(8,10))
axes[0].imshow(anAnt)
axes[0].set_title("An ant")

axes[1].imshow(aBee)
axes[1].set_title("A Bee");

print(f"Ant image shape: {anAnt.shape}")
print(f"Bee image shape: {aBee.shape}")
print(f"min & max in ant image: {anAnt.min(), anAnt.max()}")

As you can see, the images are colour, having three channels (RGB), they are also unsigned 8 bit integers (range 0-255). These two are 333x500 pixels (hxw), but having looked through the dataset, not all on them are this size: we will have to deal with that later because a neural network expects images of a constant size during training.

Have a look for other image names by running the cell below (it just lists the images in the training data folders). If you want, you can visualise some different images to see what we will be training on.

Now we are ready to make a start preparing to train a neural network. First, we must set up some infrastructure to load and transform the images.

Fortunately, we don't have to do much of this ourselves. Keras provides a class that can apply transformations automatically, including some that artificially extend the dataset, called data augmentation. To begin with, we will just resize and crop the images to the size expected by the CNN, which is 224x224 pixels.

We also need to do some normalisation on the images. This functionality is provided by the same library we will use to load a pretrained model, but we will have to add it to the keras data generator as an extra preprocessing function. Therefore, in the next cell we will:

- Load ImageDataGenerator, which does the image loading and most of the transformations
- Create a custom data generator wrapper to crop images to 224x224
- Load the pretrained model and its preprocessing function
- Put everything together to create our image loading pipeline

For now, we will leave the network alone and build it later, we mainly just want its preprocessing function right now. We are going to use a small model called ResNet18, it should be very quick to train, and give pretty good results.

In [ ]:
!ls -lh ./hymenoptera_data/train/bees 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator ## Does most of the work for us
from classification_models.keras import Classifiers
from functools import partial
import numpy as np

dataDirectory = "./hymenoptera_data/train/"

def centreCrop224(image):
    """
    Centre crops an image to 224x224. Expects the image to be 256x256 coming in.
    """
    if image.shape[1:] != (256,256,3):
        raise RuntimeError("Image is not reshaped")
    return image[:,16:-16, 16:-16, :]

def chainFunctions(image, funclist=[]):
    for func in funclist:
        image = func(image)
    return image

def cropGenerator(batchGen):
    """
    Wrap the default generator, and do centre cropping
    """
    while True:
        batch_x, batch_y = next(batchGen)
        yield(centreCrop224(batch_x), batch_y)


## quickly test the centre cropping function
test = np.zeros((1,256,256,3))
test_cropped = centreCrop224(test)
assert(test_cropped.shape == (1,224,224,3))

## now load the pretrained model class, and its preprocessing function. NB: don't create a model yet
ResNet18, preprocess_input_rn18 = Classifiers.get("resnet18")

## Now make a list of the preprocessing functions we want to apply, in order
preprocessing = [centreCrop224, preprocess_input_rn18]

## Now construct the generator object. Note that it doesn't yet link up with the data!
dataGenerator = ImageDataGenerator(validation_split=0.25, 
                                   preprocessing_function=preprocess_input_rn18, 
                                   ) ## add augmentations here later!

## Link the generator to the data
batch_size = 16
trainGen_intermediate = dataGenerator.flow_from_directory(dataDirectory, batch_size=batch_size, subset="training")
trainGen = cropGenerator(trainGen_intermediate)
valGen_intermediate = dataGenerator.flow_from_directory(dataDirectory, batch_size=batch_size, subset="validation")
valGen = cropGenerator(valGen_intermediate)

trainSamples = len(trainGen_intermediate.filenames)
valSamples = len(valGen_intermediate.filenames)

You may have spotted that we have some stuff in the data generator definition about 'validation' and 'validation splits': didn't we already do this?

This is an example of naming conventions not being agreed on yet. The validation data in the ./.../validation/... directory is also referred to as a test set. The test set is the one we must leave alone until the very end. A validation set (in keras terms) is a dataset we use along the way to steer the training of a network. In this case, we split off a little bit of the training data and use that to guide the training of the network as we go along; this helps stop us from ending up in a local minimum where the training accuracy is very good, but the network doesn't generalise and can't classify the test dataset.

With the data generator pipelines all set up, lets generate some batches and see what our data looks like!

In [ ]:
## handy label to name decoder
labelDecode = {0:"Ant", 1:"Bee"}

## generate a batch
aBatchImages, aBatchLabels = next(trainGen)

## Show some images!
fig, axes = plt.subplots(2,2, figsize=(16,16))
axes[0,0].imshow(aBatchImages[0,...].astype(np.uint8))
axes[0,0].set_title(labelDecode[np.argmax(aBatchLabels[0])])

axes[0,1].imshow(aBatchImages[1,...].astype(np.uint8))
axes[0,1].set_title(labelDecode[np.argmax(aBatchLabels[1])])

axes[1,0].imshow(aBatchImages[2,...].astype(np.uint8))
axes[1,0].set_title(labelDecode[np.argmax(aBatchLabels[2])])

axes[1,1].imshow(aBatchImages[3,...].astype(np.uint8))
axes[1,1].set_title(labelDecode[np.argmax(aBatchLabels[3])]);

Now we've got the images loading and being correctly preprocessed, we can turn our attention to constructing a network.

Remember that we are using a pretrained model, whose weights come from classifying images into one of 1000 classes. Those weights are coefficients in kernels that are used to do convolutions on the image and extract features. For a good explanation of what is going on in convolutions, have a look at the materials [here](http://www.google.co.uk). The convolutional parts of a CNN are performing 'feature extraction'. Feature extraction is the process where we take a 2D image and find a representation of it in a higher dimensional space such that we can classify images by finding the difference between them in that high dimensional space.

In a CNN, the usual way this works is with one or two fully connected layers in the classifier, which take features from the convolutional part. We will use this method, though it is also possible to extract features with a CNN and do classification with a 'classical' algortihm such as a Random Forest classifier.

For now, we need to make the following modifications to our network:

- Make sure the input size is correct (224x224x3)
- Change the 'top' (or 'classifier') of the network to classify into two classes
- Link up the main bit of the network with the classifier.

We will try two approaches to transfer learn:

1. We will use the resnet18 network to extract features, which we will then use to classify the images
2. We will use the resnet18 features as the starting point to train our own feature extraction and classification in one.

When using the network for feature extraction, we want to 'freeze' the convolutional layers, whereas when training our own specialised feature extraction, we want to train all the layers in the network. Here we will write a function that allows us to create models with either specification, and the correct number of classes.



In [ ]:
from classification_models.keras import Classifiers
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model

## Create the class definition for ResNet18
ResNet18, preprocess_input_rn18 = Classifiers.get("resnet18")


def resnetHymenoptra(input_shape=(224,224,3), classes=2, freezeLayers=True, pretrained=True):
    """
    Build a model based on resnet 18 with pretrained weights. Optionally freeze the feature extraction layers
    """
    if pretrained:
        resNetBase = ResNet18(input_shape=input_shape, weights='imagenet', include_top=False)
    else:
        resNetBase = ResNet18(input_shape=input_shape, weights=None, include_top=False)

    if freezeLayers:
        for layer in resNetBase.layers:
            layer.trainable = False
    ## construct the last bit of the model
    x = resNetBase.output
    x = GlobalAveragePooling2D()(x)
    
    prediction = Dense(classes, activation='softmax')(x)
    
    ## Now create the hymenoptera model
    rnh_model = Model(inputs=resNetBase.inputs, outputs=prediction)
    return rnh_model
    

Now we can create the model and get a summary of it. Let's start by getting a model with frozen feature extraction layers

In [ ]:
rnh_frozen = resnetHymenoptra(input_shape=(224,224,3), classes=2, freezeLayers=True, pretrained=True)

print(rnh_frozen.summary())

You should see a huge output listing. This is the resnet18 architecture. It is made out of convolutional layers (Conv2D), batch normalisation layers (BatchNormalization), activations, padding, pooling, addition and one dense layer.

At the bottom, you can see how many parameters there are in the model. When I run this, I see about 11 million in total, with 1026 being trainable. This is because we froze the entire feature extraction part of the network.

Now we can move on to training the model.



To train the model, we need to get a couple of things ready. We need to know how good the model is at classifying images, and we need a way to improve the model's perfromance.

The first is solved by using a loss function (sometimes called a cost function). A loss function tells us how close the model's prediction is to the observed labels on our dataset. There are a lot of losses, but the most appropriate in our case is either the binary_crossentropy loss, or the categorical_crossentropy loss. These two losses tell us how similar the distribution of labels given by the model and those in the observations are. The ideal value for both is zero, so we are looking for the loss value to go down and approach zero. 

Our classification problem is binary (the picture is either a bee or an ant), but because of the way we set up our generator (see the documentation (here)[https://keras.io/preprocessing/image/#imagedatagenerator-class] to see what I mean) we will start with the categorical crossentropy loss. We can try changing this part of the training later to see if we get better performance.

The second part, in which we get the model to improve, is handled by an optimizer. Optimizers are a complex field where much research is still being done. Fortunately, keras provides some very good optimizers out of the box. All optimizers try to figure out the best network parameters by looking at the gradient of the loss function with respect to those parameters. This is an 11 million dimensional partial differential equation in our case, so it is very much not trivial! 

Optimizers generally work on batches of data drawn from the whole dataset. This means that the gradient is approximage at every evaluation, so we see some noise in the training performance. Optimizers also have a learning rate parameter, which controls how quickly they step along the direction of decreasing gradient. this can be very important, as steps too large can end up oscillating around a minimum rather than converging to it.

The choice of optimizer and its settings can have a profound effect on how well a network trains and generalises, so it is a good idea to try a few. This process is called hyperparameter tuning.

### Hyperparameters
A hyperparameter is something which indirectly influences the performance of a network. For example, the choice of optimizer may have an impact on how well a network generalises to an unseen test set. There are many hyperparameters, some of which are more subtle than others. Here are a few we can try playing with here:

- Batch size. I set it to 16 where we created the image generators. You could go higher or lower.
- Learning rate. I have used a default of 1E-3. You could try higher or lower.
- Optimizer. I chose stochastic gradient descent with momentum. There are a lot of optimizers in keras whose documentation you can find here: https://keras.io/optimizers/. You can also try changing the momentum value.
- Data augmentations. We haven't done any, so anythig you add can change the performance drastically.
- Loss function. We're using categorical crossentrpy
- Anything else you think might affect the model!

Before trying anything fancy, let's just see if we can train the classifier using the feature extraction from resnet 18. 

### Training 
To run the training in keras is very simple. We:

1. Compile the model. This links up the model, loss function and optimizer (and any metrics we want)
2. Use the model.fit_generator function with our image data generators from earlier

In [ ]:
## first, compile the model
from keras import optimizers

## define the optimizer
opt = optimizers.SGD(lr=1E-3, momentum=0.9)

rnh_frozen.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])


In [ ]:
## Now fit it

steps_train = trainSamples // batch_size
steps_val = valSamples // batch_size

frozen_fit_history = rnh_frozen.fit_generator(trainGen,
                                              verbose=1,
                                              steps_per_epoch=steps_train, 
                                              epochs=5, 
                                              validation_data=valGen, 
                                              validation_steps=steps_val
                                             )

In [ ]:
plt.plot(frozen_fit_history.history['acc'], label="Training")
plt.plot(frozen_fit_history.history['val_acc'], label="Validation")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Accuracy");

For me, I get something like 87% accuracy in just a few epochs of training, which is great!

Now though, we want to see if we can push the accuracy higher by finetuning the convolution parts of the network. To do this, lets create a new model, but with the pretrained weights un-frozen. We will then compile with the same options, but a much lower learning rate, and try fitting that model

In [ ]:
rnh_defrosted =  resnetHymenoptra(input_shape=(224,224,3), classes=2, freezeLayers=False, pretrained=True)

print(rnh_defrosted.summary())

opt2 = optimizers.SGD(lr=1e-4, momentum=0.9, nesterov=False)
rnh_defrosted.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

steps_train = trainSamples // batch_size
steps_val = valSamples // batch_size

unfrozen_fit_history = rnh_defrosted.fit_generator(trainGen, 
                                              steps_per_epoch=steps_train, 
                                              epochs=25, 
                                              validation_data=valGen, 
                                              validation_steps=steps_val)


plt.plot(unfrozen_fit_history.history['acc'], label="Training")
plt.plot(unfrozen_fit_history.history['val_acc'], label="Validation")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Accuracy");

At this point, we can test our model on the test set. To do this, we create another image data generator that loads the test set. It will have exactly the same parameters as the training and validation generators

In [ ]:
testDirectory = "./hymenoptera_data/val/"
testGenBase = ImageDataGenerator(preprocessing_function=preprocess_input_rn18) ## no augmentation for now
testGen = cropGenerator(testGenBase.flow_from_directory(testDirectory, batch_size=1))

## model shootout
print(rnh_frozen.evaluate_generator(testGen, steps=153))
print(rnh_defrosted.evaluate_generator(testGen, steps=153))


So hopefully we've got some networks that are pretty good at predicting whether an image is of a bee or an ant. Let's try them on some 'real' data.

Can one of these networks tell me if I'm dealing with an africanized honeybee? Let's find out

In [ ]:
## Download an image
!wget -q https://modernfarmer.com/wp-content/uploads/2016/06/africanizedbee-1200x742.jpg
!wget -q https://upload.wikimedia.org/wikipedia/commons/4/4d/Apis_mellifera_Western_honey_bee.jpg
!wget -q https://www.dogstrust.org.uk/dogimages/1238053_suzie_20200225014216_img_0154_253.JPG
    
## load it, resize to 256x256 and centre crop
from skimage.io import imread
from skimage.transform import resize

scaryBee = imread("Apis_mellifera_Western_honey_bee.jpg")
scaryBee = resize(scaryBee, (256,256), preserve_range=True)
scaryBee = centreCrop224(scaryBee[np.newaxis,...])
print(scaryBee.max())
plt.imshow(scaryBee[0,...].astype(np.uint8))


prediction_frozen = rnh_frozen.predict(scaryBee)
prediction_defrost = rnh_defrosted.predict(scaryBee)

print(prediction_frozen, prediction_defrost)
print(labelDecode[np.argmax(prediction_frozen)], labelDecode[np.argmax(prediction_defrost)])




That's pretty good!

Play around and try seeing what happens if you show the network images of ants. What happens if you give it an image of a dog?

## Next steps
Now you've got a pipeline to train a network, you can try exploring some of the hyperparameters mentioned earlier. One of the first things you could try is to add image augmentations to see if you can push the accuracy higher. You could also try other optimisers, for example Adam is a pretty good one.

### Loss function
Remember, this task is binary classification. Why then are we treating it like multi-class classification? We could switch over to use binary crossentropy as the loss. This would require the following changes in the model:

1. Change the output activation to 'sigmoid' rather than 'softmax'
2. Add the keyword argument when creating the data generator `class_mode='binary'`
3. Have `classes=1` when you create the networks.

Try these changes, thay might improve accuracy, or they might not. 

You can also try other loss functions that are applicable to categorical data. Have a look [here](https://keras.io/losses/) for other loss functions you could try.

## Possible extension task - why did the network say that?
Interpreting the output of your neural network can be very tricky. Why does it think this image of a dog is actually a bee? If is correctly classified an image, which bit of the image was most important?

We can explore these questions using a technique called Gradient-Class Activation Mapping, or Grad-CAM. Grad-CAM tries to find which bit of an image was the most influential when classifying it. The mechanics of how it works are a bit tricky to explain here, but if you're interested have a look at the paper.

As with everything else in python, someone has implemented a Grad-CAM library for us! We will now use that library to try to probe what our networks are doing.

In [ ]:
!pip3 install git+https://github.com/raghakot/keras-vis.git

In [ ]:
import matplotlib.cm as cm
from vis.visualization import overlay, visualize_cam
from vis.utils import utils
# from vis.visualization import visualize_cam, overlay

penultimate_layer = utils.find_layer_idx(rnh_defrosted, 'stage4_unit2_conv2')
layer_idx = utils.find_layer_idx(rnh_defrosted, 'dense_2')

grads = visualize_cam(rnh_defrosted, layer_idx, 
                      penultimate_layer_idx=penultimate_layer, 
                      filter_indices=1, 
                      seed_input=scaryBee, 
                      backprop_modifier=None)
heatmap = np.uint8(cm.jet(grads)[...,:3] * 255)

fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.imshow(overlay(heatmap, scaryBee[0,...]));
plt.axis('off');